In [1]:
from utils import set_seed
import torch
import os
import logging
import argparse
from trainer import SimcseTrainer
from dataset import DATASET_MAPPING_DICT
from utils import PreprocessorFactory 
from utils import get_model_argparse
from model import MODEL_MAPPING_DICT
from model import CONFIG_MAPPING_DICT
from logger import Experi_Logger
from transformers import (
    AdamW,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
)

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from config.nli_config import nli_parser_model_args

In [3]:
args = nli_parser_model_args()

In [4]:
from logger import Experi_Logger

In [5]:
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.output_dir = f'/app/data/model/{args.pretrained_model}'
args.log_dir = f'/app/data/log/{args.pretrained_model}'
args.experiments_path = f'/app/data/experiment/{args.pretrained_model}/experiment.csv'
args.model_max_len = 50
args.is_preprocessed = True
args.valid_first = False
args.data_type='nli'

In [6]:
#args.amp_use

# load pretrained model 

In [7]:
args.pretrained_model

'klue/roberta-large'

In [ ]:
#model = AutoModel.from_pretrained(args.pretrained_model)
model = MODEL_MAPPING_DICT['sent_roberta'].from_pretrained(
    args.pretrained_model, **vars(args), 
)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

You are using a model of type roberta to instantiate a model of type sent_roberta. This is not supported for all configurations of models and can yield errors.


# load data

In [ ]:
data_path = '/app/data/open_data/KorNLI/multinli.train.ko.tsv'

In [ ]:
preprocessor = PreprocessorFactory(data_type=args.data_type)

In [ ]:
nli_data = preprocessor.preprocess(
                              data_path=data_path,
                              tokenizer=tokenizer, 
                              save_path=None
)

In [ ]:
preprocessor = PreprocessorFactory('sts')

In [ ]:
data_path = '/app/data/open_data/KorSTS/sts-train.tsv'
sts_data = preprocessor.preprocess(
                              data_path=data_path,
                              tokenizer=tokenizer, 
                              save_path=None
)

In [ ]:
train_dataset = DATASET_MAPPING_DICT['Unsup_simcse']
train_dataset = train_dataset(args=args, features=nli_data, max_length=args.model_max_len, tokenizer=tokenizer)

In [ ]:
val_dataset = DATASET_MAPPING_DICT['StsDataset']
val_dataset = val_dataset(args=args, features=sts_data, max_length=args.model_max_len, tokenizer=tokenizer)

# Trainer

In [ ]:
from trainer import SimcseTrainer
import torch.cuda.amp as amp

In [ ]:
trainer = SimcseTrainer(args=args, logger=logging)

In [ ]:
trainer.model_setting(model_type=args.model_type, train_dataset=train_dataset, model=model, tokenizer=tokenizer)

In [ ]:
trainer.train(model=model, tokenizer=tokenizer, train_dataset=train_dataset, val_dataset=val_dataset, model_type=args.model_type)

In [ ]:
!nvidia-smi